In [ ]:
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm
import multiprocessing,os
import pandas as pd
from stellargraph import StellarGraph
from stellargraph import datasets
from stellargraph import IndexedArray

In [ ]:
!pip install git+https://github.com/VenkateshwaranB/stellargraph.git

In [ ]:
basic_repo='./facebook_react'

users = pd.read_csv(basic_repo+'/index/user_index.txt', sep='\t', header=None, names=['user_id', 'user'], keep_default_na=False, encoding='utf-8')
users= list(map(lambda x:x, users['user_id'].tolist()))
repos= pd.read_csv(basic_repo+'/index/repo_index.txt', sep='\t', header=None, names=['repo_id', 'repo'], keep_default_na=False, encoding='utf-8')
repos= list(map(lambda x:x+len(users), repos['repo_id'].tolist()))
issues =pd.read_csv(basic_repo+'/index/issue_index.txt', sep='\t', header=None, names=['issue_id', 'issue'], keep_default_na=False, encoding='utf-8')
issues= list(map(lambda x:x+len(users)+len(repos), issues['issue_id'].tolist()))
prs = pd.read_csv(basic_repo+'/index/pr_index.txt', sep='\t', header=None, names=['pr_id', 'pr'], keep_default_na=False, encoding='utf-8')
prs= list(map(lambda x:x+len(users)+len(repos)+len(issues), prs['pr_id'].tolist()))

issues_ = IndexedArray(index=issues)
prs_ = IndexedArray(index=prs)
users_ = IndexedArray(index=users)
repos_ = IndexedArray(index=repos)

user_repo=pd.read_csv(basic_repo+'/edge_index/user_repo_index.txt', sep='\t', header=None, names=['user_id', 'repo_id'], keep_default_na=False, encoding='utf-8')
user_issue=pd.read_csv(basic_repo+'/edge_index/user_issue_index.txt', sep='\t', header=None, names=['user_id', 'issue_id'], keep_default_na=False, encoding='utf-8')
user_pr=pd.read_csv(basic_repo+'/edge_index/user_pr_index.txt', sep='\t', header=None, names=['user_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

repo_repo = pd.read_csv(basic_repo+'/edge_index/repo_repo_index.txt', sep='\t', header=None, names=['repo_id', 'repo_id_1'], keep_default_na=False, encoding='utf-8')
repo_issue = pd.read_csv(basic_repo+'/edge_index/repo_issue_index.txt', sep='\t', header=None, names=['repo_id', 'issue_id'], keep_default_na=False, encoding='utf-8')
repo_pr = pd.read_csv(basic_repo+'/edge_index/repo_pr_index.txt', sep='\t', header=None, names=['repo_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

issue_issue=pd.read_csv(basic_repo+'/edge_index/issue_issue_index.txt', sep='\t', header=None, names=['issue_id', 'issue_id_1'], keep_default_na=False, encoding='utf-8')
pr_pr=pd.read_csv(basic_repo+'/edge_index/pr_pr_index.txt', sep='\t', header=None, names=['pr_id', 'pr_id_1'], keep_default_na=False, encoding='utf-8')
issue_pr=pd.read_csv(basic_repo+'/edge_index/issue_pr_index.txt', sep='\t', header=None, names=['issue_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

edges_UR = pd.DataFrame({"source": user_repo['user_id'].tolist(), "target":list(map(lambda x:x+len(users),user_repo['repo_id'].tolist()))},index=list(range(0,len(user_repo))))
edges_UI=  pd.DataFrame({"source": user_issue['user_id'].tolist(), "target":list(map(lambda x:x+len(users)+len(repos),user_issue['issue_id'].tolist()))},index=list(range(len(user_repo),len(user_repo)+len(user_issue))))
edges_UP=  pd.DataFrame({"source": user_pr['user_id'].tolist(), "target":list(map(lambda x:x+len(users)+len(repos)+len(issues),user_pr['pr_id'].tolist()))},index=list(range(len(user_repo)+len(user_issue),len(user_repo)+len(user_issue)+len(user_pr))))

edges_RR=  pd.DataFrame({"source":list(map(lambda x:x+len(users),repo_repo['repo_id'].tolist())), "target":list(map(lambda x:x+len(users),repo_repo['repo_id_1'].tolist()))},
                        index=list(range(len(user_repo)+len(user_issue)+len(user_pr),len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo))))
edges_RI=  pd.DataFrame({"source":list(map(lambda x:x+len(users),repo_issue['repo_id'].tolist())), "target":list(map(lambda x:x+len(users)+len(repos),repo_issue['issue_id'].tolist()))},
                        index=list(range(len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo),len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue))))
edges_RP=  pd.DataFrame({"source":list(map(lambda x:x+len(users),repo_pr['repo_id'].tolist())), "target":list(map(lambda x:x+len(users)+len(repos)+len(issues),repo_pr['pr_id'].tolist()))},
                        index=list(range(len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue),len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue)+len(repo_pr))))
edges_II=  pd.DataFrame({"source":list(map(lambda x:x+len(users)+len(repos),issue_issue['issue_id'].tolist())),
                         "target":list(map(lambda x:x+len(users)+len(repos),issue_issue['issue_id_1'].tolist()))},
                        index=list(range(len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue)+len(repo_pr),len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue)+len(repo_pr)+len(issue_issue))))
edges_PP=  pd.DataFrame({"source":list(map(lambda x:x+len(users)+len(repos)+len(issues),pr_pr['pr_id'].tolist())),"target":list(map(lambda x:x+len(users)+len(repos)+len(issues),pr_pr['pr_id_1'].tolist()))},
                        index=list(range(len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue)+len(repo_pr)+len(issue_issue),
                                         len(user_repo)+len(user_issue)+len(user_pr)+len(repo_repo)+len(repo_issue)+len(repo_pr)+len(issue_issue)+len(pr_pr))))

Graph = StellarGraph(nodes={"issue": issues_, "pr": prs_,"user": users_, "repo": repos_},
                     edges={"UR":edges_UR,'UI':edges_UI,'UP':edges_UP,'RR':edges_RR,'RI':edges_RI,'RP':edges_RP,'II':edges_II,'PP':edges_PP})

metapaths = [
    [
        [["issue", "issue"]],
    [ ["issue", "repo", "issue"]],
    [ ["issue", "user", "issue"]],
    [ ["issue", "repo", "repo", "issue"]],
    [ ["issue", "repo", "user", "repo", "issue"]],
    [ ["issue", "user", "repo","user","issue"]]
     ],
    [
        [["pr", "pr"]],
    [["pr", "repo", "pr"]],
    [["pr", "user", "pr"]],
    [["pr", "repo", "repo", "pr"]],
    [["pr", "repo", "user", "repo", "pr"]],
    [["pr", "user", "repo", "user", "pr"]]],
]


from stellargraph.data import UniformRandomMetaPathWalk

# Create the random walker
rw = UniformRandomMetaPathWalk(Graph)

walk_length = 8

issue_root_nodes=list(map(lambda x:x+len(users)+len(repos),issue_pr['issue_id'].tolist()))
pr_root_nodes=list(map(lambda x:x+len(users)+len(repos)+len(issues),issue_pr['pr_id'].tolist()))
i=0
for metapath in metapaths[1]:
  print(metapath)
  i+=1
  walks = rw.run(
      nodes=list(pr_root_nodes),  # root nodes
      length= walk_length,  # maximum length of a random walk
      n=1,  # number of random walks per root node
      metapaths=metapath,  # the metapaths
  )

  walks=np.array(walks)
  np.save('/content/drive/MyDrive/facebook_react/metapath/pr/metapath_pr'+str(i)+'.npy',walks)


[['pr', 'pr']]


<ipython-input-21-e2127a85b666>:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  walks=np.array(walks)


[['pr', 'repo', 'pr']]
[['pr', 'user', 'pr']]
[['pr', 'repo', 'repo', 'pr']]


<ipython-input-21-e2127a85b666>:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  walks=np.array(walks)


[['pr', 'repo', 'user', 'repo', 'pr']]
[['pr', 'user', 'repo', 'user', 'pr']]


<ipython-input-21-e2127a85b666>:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  walks=np.array(walks)


In [ ]:
from gensim.models import Word2Vec
import os
import numpy as np

issue_walks=[]
# pr_walks=[]

# for index in os.listdir('/content/drive/MyDrive/facebook_react/metapath/pr/'):

#   array_a=np.load('/content/drive/MyDrive/facebook_react/metapath/pr/'+index,allow_pickle=True)
#   for i in array_a.tolist():
#     pr_walks.append(i)
# print(len(pr_walks))
# str_pr_walks = [[str(n) for n in walk] for walk in pr_walks]

# pr_model = Word2Vec(str_pr_walks, vector_size=64, window=5, min_count=0, sg=1, workers=2, epochs=100)
# pr_model.save('/content/drive/MyDrive/facebook_react/metapath/pr/pr_word2vec.model')

# vectors = pr_model.wv.vectors #获取模型中全部的节点向量
# words = pr_model.wv.index_to_key #获取模型中全部的节点类型

for index in os.listdir('/content/drive/MyDrive/facebook_react/metapath/issue/'):

  array_a=np.load('/content/drive/MyDrive/facebook_react/metapath/issue/'+index,allow_pickle=True)
  for i in array_a.tolist():
    issue_walks.append(i)

str_issue_walks = [[str(n) for n in walk] for walk in issue_walks]

issue_model = Word2Vec(str_issue_walks, vector_size=64, window=5, min_count=0, sg=1, workers=2, epochs=100)
issue_model.save('/content/drive/MyDrive/facebook_react/metapath/issue/issue_word2vec.model')

vectors = issue_model.wv.vectors #获取模型中全部的节点向量
words = issue_model.wv.index_to_key #获取模型中全部的节点类型

In [ ]:
import pandas as pd
import torch
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics import precision_score, recall_score, accuracy_score
auc_list = []
ap_list = []
ar_list = []
af1_list = []
issue_model = Word2Vec.load('/content/drive/MyDrive/facebook_react/metapath/issue/issue_word2vec.model')
pr_model = Word2Vec.load('/content/drive/MyDrive/facebook_react/metapath/pr/pr_word2vec.model')
issue_vectors = issue_model.wv.vectors #获取模型中全部的节点向量
issue_words = issue_model.wv.index_to_key #获取模型中全部的节点类型
pr_vectors = pr_model.wv.vectors #获取模型中全部的节点向量
pr_words = pr_model.wv.index_to_key #获取模型中全部的节点类型

basic_repo='/content/drive/MyDrive/facebook_react'
issue_pr=pd.read_csv(basic_repo+'/edge_index/issue_pr_index.txt', sep='\t', header=None, names=['issue_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

train_val_test_pos_issue_pr = np.load(basic_repo + '/train_val_test_idx.npz')
train_val_test_neg_issue_pr = np.load(basic_repo + '/train_val_test_neg_issue_pr.npz')

train_pos_issue_pr = train_val_test_pos_issue_pr['train_pos_issue_pr']
val_pos_issue_pr = train_val_test_pos_issue_pr['val_pos_issue_pr']
test_pos_issue_pr = train_val_test_pos_issue_pr['test_pos_issue_pr']
train_neg_issue_pr = train_val_test_neg_issue_pr['train_neg_issue_pr']
val_neg_issue_pr = train_val_test_neg_issue_pr['val_neg_issue_pr']
test_neg_issue_pr = train_val_test_neg_issue_pr['test_neg_issue_pr']

predict=[]
for index in train_pos_issue_pr:
  pos_embedding_user=np.array([0]*64)
  pos_embedding_artist=np.array([0]*64)
  if str(index[0]) in issue_words:
    pos_embedding_user=issue_model.wv[str(index[0])]
  if str(index[1]) in pr_words:
    pos_embedding_artist=pr_model.wv[str(index[1])]

  # pos_embedding_user = pos_embedding_user.view(-1, 1, pos_embedding_user.shape[1])
  # pos_embedding_artist = pos_embedding_artist.view(-1, pos_embedding_artist.shape[1], 1)
  pos_out = np.dot(pos_embedding_user, pos_embedding_artist)
  # pos_out = pos_out.squeeze(dim=-1).float()

  predict.append(pos_out)

for index in train_neg_issue_pr:
  neg_embedding_user=np.array([0]*64)
  neg_embedding_artist=np.array([0]*64)
  if str(index[0]) in issue_words:
    neg_embedding_user=issue_model.wv[str(index[0])]
  if str(index[1]) in pr_words:
    neg_embedding_artist=pr_model.wv[str(index[1])]

  # neg_embedding_user = neg_embedding_user.view(-1, 1, neg_embedding_user.shape[1])
  # neg_embedding_artist = neg_embedding_artist.view(-1, neg_embedding_artist.shape[1], 1)
  neg_out = np.dot(neg_embedding_user, neg_embedding_artist)
  # neg_out = neg_out.squeeze(dim=-1).float()
  predict.append(neg_out)

predict=np.array(predict)
predict=np.where(predict>=0.5,1,0)
y_true_test=np.array([1] * len(train_pos_issue_pr) + [0] * len(train_neg_issue_pr))

auc= accuracy_score(y_true_test, predict.astype(int))
precision= precision_score(y_true_test, predict)
recall= recall_score(y_true_test, predict)

auc_list.append(auc)
ap_list.append(precision)
ar_list.append(recall)
af1_list.append(2*precision*recall/(precision+recall))

print('----------------------------------------------------------------')
print('Link Prediction Tests Summary')
print('AUC_mean = {}'.format(np.mean(auc_list)))
print('AP_mean = {}'.format(np.mean(ap_list)))
print('AR_mean = {}'.format(np.mean(ar_list)))
print('AF1_mean = {},'.format(np.mean(af1_list)))

----------------------------------------------------------------
Link Prediction Tests Summary
AUC_mean = 0.2999459167117361
AP_mean = 0.7199124726477024
AR_mean = 0.09572301425661914
AF1_mean = 0.1689779147406266,


In [ ]:
import numpy as np
import pandas as pd
import pickle,pathlib,scipy.sparse

basic_repo='/content/drive/MyDrive/facebook_react'
save_path='/content/drive/MyDrive/facebook_react/processed/'

# user_repo=pd.read_csv(basic_repo+'/edge_index/user_repo_index.txt', sep='\t', header=None, names=['user_id', 'repo_id'], keep_default_na=False, encoding='utf-8')
# user_issue=pd.read_csv(basic_repo+'/edge_index/user_issue_index.txt', sep='\t', header=None, names=['user_id', 'issue_id'], keep_default_na=False, encoding='utf-8')
# user_pr=pd.read_csv(basic_repo+'/edge_index/user_pr_index.txt', sep='\t', header=None, names=['user_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

# repo_repo = pd.read_csv(basic_repo+'/edge_index/repo_repo_index.txt', sep='\t', header=None, names=['repo_id', 'repo_id_1'], keep_default_na=False, encoding='utf-8')
# repo_issue = pd.read_csv(basic_repo+'/edge_index/repo_issue_index.txt', sep='\t', header=None, names=['repo_id', 'issue_id'], keep_default_na=False, encoding='utf-8')
# repo_pr = pd.read_csv(basic_repo+'/edge_index/repo_pr_index.txt', sep='\t', header=None, names=['repo_id', 'pr_id'], keep_default_na=False, encoding='utf-8')

# issue_issue=pd.read_csv(basic_repo+'/edge_index/issue_issue_index.txt', sep='\t', header=None, names=['issue_id', 'issue_id_1'], keep_default_na=False, encoding='utf-8')
# # issue_pr=pd.read_csv(basic_repo+'/edge_index/issue_pr_index.txt', sep='\t', header=None, names=['issue_id', 'pr_id'], keep_default_na=False, encoding='utf-8')
# pr_pr=pd.read_csv(basic_repo+'/edge_index/pr_pr_index.txt', sep='\t', header=None, names=['pr_id', 'pr_id_1'], keep_default_na=False, encoding='utf-8')

# issues = pd.read_csv(basic_repo+'/index/issue_index.txt', sep='\t', header=None, names=['issue_id', 'issue'], keep_default_na=False, encoding='utf-8')
# prs = pd.read_csv(basic_repo+'/index/pr_index.txt', sep='\t', header=None, names=['pr_id', 'pr'], keep_default_na=False, encoding='utf-8')
# users = pd.read_csv(basic_repo+'/index/user_index.txt', sep='\t', header=None, names=['user_id', 'user'], keep_default_na=False, encoding='utf-8')
# repos= pd.read_csv(basic_repo+'/index/repo_index.txt', sep='\t', header=None, names=['repo_id', 'repo'], keep_default_na=False, encoding='utf-8')

# num_users=len(users)
# num_repos=len(repos)
# num_issues=len(issues)
# num_prs=len(prs)

# dim = len(users) + len(repos) + len(issues) + len(prs)

# ########### 保存每个节点的type all nodes (users, repos, issues and prs) type labels
# type_mask = np.zeros((dim), dtype=int)
# type_mask[len(users):len(users)+len(repos)] = 1
# type_mask[len(users)+len(repos):len(users)+len(repos)+len(issues)] = 2
# type_mask[len(users)+len(repos)+len(issues):] = 3


# #将每一个节点对应 的标号映射到列表的位置上
# user_id_mapping = {row['user_id']: i for i, row in users.iterrows()}
# repo_id_mapping = {row['repo_id']: i + len(users) for i, row in repos.iterrows()}
# issue_id_mapping = {row['issue_id']: i + len(users)+len(repos) for i, row in issues.iterrows()}
# pr_id_mapping = {row['pr_id']: i + len(users)+len(repos)+len(issues) for i, row in prs.iterrows()}

####### 构建邻接矩阵

# adjM_ur = np.zeros((dim, dim), dtype=int)
# for _, row in user_repo.iterrows():
#     idx1 = user_id_mapping[row['user_id']]
#     idx2 = repo_id_mapping[row['repo_id']]
#     adjM_ur[idx1, idx2] = 1
# adjM_ur=scipy.sparse.csr_matrix(adjM_ur)
# adjM_ru = adjM_ur.transpose()

# adjM_ui = np.zeros((dim, dim), dtype=int)
# for _, row in user_issue.iterrows():
#     idx1 = user_id_mapping[row['user_id']]
#     idx2 = issue_id_mapping[row['issue_id']]
#     adjM_ui[idx1, idx2] = 1
# adjM_iu=adjM_ui.transpose()

# adjM_up = np.zeros((dim, dim), dtype=int)
# for _, row in user_pr.iterrows():
#     idx1 = user_id_mapping[row['user_id']]
#     idx2 = pr_id_mapping[row['pr_id']]
#     adjM_up[idx1, idx2] = 1
# adjM_pu = adjM_up.transpose()

# adjM_rr = np.zeros((dim, dim), dtype=int)
# for _, row in repo_repo.iterrows():
#     idx1 = repo_id_mapping[row['repo_id']]
#     idx2 = repo_id_mapping[row['repo_id_1']]
#     adjM_rr[idx1, idx2] = 1
# adjM_rr_t = adjM_rr.transpose()

# adjM_ri = np.zeros((dim, dim), dtype=int)
# for _, row in repo_issue.iterrows():
#     idx1 = repo_id_mapping[row['repo_id']]
#     idx2 = issue_id_mapping[row['issue_id']]
#     adjM_ri[idx1, idx2] = 1
# adjM_ir=adjM_ri.transpose()

# adjM_rp = np.zeros((dim, dim), dtype=int)
# for _, row in repo_pr.iterrows():
#     idx1 = repo_id_mapping[row['repo_id']]
#     idx2 = pr_id_mapping[row['pr_id']]
#     adjM_rp[idx1, idx2] = 1
# adjM_pr=adjM_rp.transpose()

# adjM_ii=np.zeros((dim, dim), dtype=int)
# for _, row in issue_issue.iterrows():
#     idx1 = issue_id_mapping[row['issue_id']]
#     idx2 = issue_id_mapping[row['issue_id_1']]
#     adjM_ii[idx1, idx2] = 1
# adjM_ii_t=adjM_ii.transpose()

# adjM_pp=np.zeros((dim, dim), dtype=int)
# for _, row in pr_pr.iterrows():
#     idx1 = pr_id_mapping[row['pr_id']]
#     idx2 = pr_id_mapping[row['pr_id_1']]
#     adjM_pp[idx1, idx2] = 1
# adjM_pp_t=adjM_pp.transpose()

# scipy.sparse.save_npz(save_path + 'adjM_ur.npz', scipy.sparse.csr_matrix(adjM_ur))
# scipy.sparse.save_npz(save_path + 'adjM_ru.npz', scipy.sparse.csr_matrix(adjM_ru))

# scipy.sparse.save_npz(save_path + 'adjM_ui.npz', scipy.sparse.csr_matrix(adjM_ui))
# scipy.sparse.save_npz(save_path + 'adjM_iu.npz', scipy.sparse.csr_matrix(adjM_iu))

# scipy.sparse.save_npz(save_path + 'adjM_up.npz', scipy.sparse.csr_matrix(adjM_up))
# scipy.sparse.save_npz(save_path + 'adjM_pu.npz', scipy.sparse.csr_matrix(adjM_pu))

# scipy.sparse.save_npz(save_path + 'adjM_rr.npz', scipy.sparse.csr_matrix(adjM_rr))
# scipy.sparse.save_npz(save_path + 'adjM_rr_t.npz', scipy.sparse.csr_matrix(adjM_rr_t))

# scipy.sparse.save_npz(save_path + 'adjM_ri.npz', scipy.sparse.csr_matrix(adjM_ri))
# scipy.sparse.save_npz(save_path + 'adjM_ir.npz', scipy.sparse.csr_matrix(adjM_ir))

# scipy.sparse.save_npz(save_path + 'adjM_rp.npz', scipy.sparse.csr_matrix(adjM_rp))
# scipy.sparse.save_npz(save_path + 'adjM_pr.npz', scipy.sparse.csr_matrix(adjM_pr))

# scipy.sparse.save_npz(save_path + 'adjM_ii.npz', scipy.sparse.csr_matrix(adjM_ii))
# scipy.sparse.save_npz(save_path + 'adjM_ii_t.npz', scipy.sparse.csr_matrix(adjM_ii_t))

# scipy.sparse.save_npz(save_path + 'adjM_pp.npz', scipy.sparse.csr_matrix(adjM_pp))
# scipy.sparse.save_npz(save_path + 'adjM_pp_t.npz', scipy.sparse.csr_matrix(adjM_pp_t))
adjM_ur = scipy.sparse.load_npz(save_path + 'adjM_ur.npz')
adjM_ur=adjM_ur.toarray()
# adjM_ru = scipy.sparse.load_npz(save_path + 'adjM_ru.npz')
# adjM_ru=adjM_ru.toarray()
# adjM_ui = scipy.sparse.load_npz(save_path + 'adjM_ui.npz')
# adjM_ui= adjM_ui.toarray()
# adjM_iu = scipy.sparse.load_npz(save_path + 'adjM_iu.npz')
# adjM_iu=adjM_iu.toarray()
# adjM_up = scipy.sparse.load_npz(save_path + 'adjM_up.npz')
# adjM_up= adjM_up.toarray()
# adjM_pu = scipy.sparse.load_npz(save_path + 'adjM_pu.npz')
# adjM_pu=adjM_pu.toarray()

edges=[adjM_ur]
      #adjM_ru]
      # adjM_ui,adjM_iu]
      # adjM_up,adjM_pu]
      #  adjM_rr,adjM_rr_t]
      #  adjM_ri,adjM_ir,adjM_rp,adjM_pr,adjM_ii,adjM_ii_t,adjM_pp,adjM_pp_t]

with open(basic_repo+'/edges.pkl', 'wb') as f:
    pickle.dump(edges, f)